# 🔄 Convert LoRA to OpenVINO (Colab)

Convert Vietnamese LoRA to OpenVINO format on Colab (free disk space!)

## 📦 Step 1: Install Dependencies

In [ ]:
!pip install -q torch torchvision diffusers transformers accelerate safetensors openvino onnx
!pip install -q --upgrade diffusers transformers

## 📤 Step 2: Upload LoRA File

Upload file `pytorch_lora_weights.safetensors` từ folder `lora_vietnamese/`

In [ ]:
from google.colab import files
import os

# Create folder
os.makedirs('lora_vietnamese', exist_ok=True)

print("Upload file: pytorch_lora_weights.safetensors")
uploaded = files.upload()

# Move to folder
for filename in uploaded.keys():
    os.rename(filename, f'lora_vietnamese/{filename}')
    print(f"✓ Uploaded: lora_vietnamese/{filename}")

## 🔧 Step 3: Conversion Script

In [ ]:
import torch
import openvino as ov
from pathlib import Path
from diffusers import StableDiffusionPipeline
import tempfile
import json
import shutil

def export_text_encoder(text_encoder, tokenizer, output_path):
    """Export CLIP text encoder"""
    print("Converting Text Encoder...")
    
    text_encoder.eval()
    text_encoder = text_encoder.float()
    
    dummy_input = torch.ones((1, 77), dtype=torch.long)
    
    with tempfile.NamedTemporaryFile(suffix=".onnx", delete=False) as tmp_file:
        torch.onnx.export(
            text_encoder,
            dummy_input,
            tmp_file.name,
            input_names=["input_ids"],
            output_names=["hidden_states"],
            dynamic_axes={
                "input_ids": {0: "batch_size"},
                "hidden_states": {0: "batch_size"}
            },
            opset_version=14,
            do_constant_folding=True
        )
        
        ov_model = ov.convert_model(tmp_file.name)
        output_path.mkdir(parents=True, exist_ok=True)
        ov.save_model(ov_model, output_path / "openvino_model.xml")
        
        config = {
            "model_type": "CLIPTextModel",
            "max_position_embeddings": text_encoder.config.max_position_embeddings,
            "vocab_size": text_encoder.config.vocab_size,
            "hidden_size": text_encoder.config.hidden_size,
        }
        
        with open(output_path / "config.json", 'w') as f:
            json.dump(config, f, indent=2)
    
    print(f"✓ Text Encoder saved")

def export_unet(unet, output_path):
    """Export UNet"""
    print("Converting UNet...")
    
    unet.eval()
    unet = unet.float()
    
    sample = torch.randn(1, 4, 64, 64, dtype=torch.float32)
    timestep = torch.tensor([1], dtype=torch.long)
    encoder_hidden_states = torch.randn(1, 77, 768, dtype=torch.float32)
    
    with tempfile.NamedTemporaryFile(suffix=".onnx", delete=False) as tmp_file:
        torch.onnx.export(
            unet,
            (sample, timestep, encoder_hidden_states),
            tmp_file.name,
            input_names=["sample", "timestep", "encoder_hidden_states"],
            output_names=["noise_pred"],
            dynamic_axes={
                "sample": {0: "batch_size", 2: "height", 3: "width"},
                "encoder_hidden_states": {0: "batch_size"},
                "noise_pred": {0: "batch_size", 2: "height", 3: "width"}
            },
            opset_version=14,
            do_constant_folding=True
        )
        
        print("  Converting ONNX to OpenVINO IR...")
        ov_model = ov.convert_model(tmp_file.name)
        
        output_path.mkdir(parents=True, exist_ok=True)
        ov.save_model(ov_model, output_path / "openvino_model.xml")
        
        config = unet.config
        if hasattr(config, 'to_dict'):
            config_dict = config.to_dict()
        else:
            config_dict = dict(config)
            
        with open(output_path / "config.json", 'w') as f:
            json.dump(config_dict, f, indent=2)
    
    print(f"✓ UNet saved")

def export_vae_encoder(vae, output_path):
    """Export VAE encoder"""
    print("Converting VAE Encoder...")
    
    vae.encoder.eval()
    vae.encoder = vae.encoder.float()
    
    dummy_input = torch.randn(1, 3, 512, 512, dtype=torch.float32)
    
    class VAEEncoderWrapper(torch.nn.Module):
        def __init__(self, encoder):
            super().__init__()
            self.encoder = encoder
            
        def forward(self, sample):
            return self.encoder(sample)
    
    wrapper = VAEEncoderWrapper(vae.encoder)
    
    with tempfile.NamedTemporaryFile(suffix=".onnx", delete=False) as tmp_file:
        torch.onnx.export(
            wrapper,
            dummy_input,
            tmp_file.name,
            input_names=["sample"],
            output_names=["latent"],
            dynamic_axes={
                "sample": {0: "batch_size", 2: "height", 3: "width"},
                "latent": {0: "batch_size", 2: "height", 3: "width"}
            },
            opset_version=14,
            do_constant_folding=True
        )
        
        ov_model = ov.convert_model(tmp_file.name)
        output_path.mkdir(parents=True, exist_ok=True)
        ov.save_model(ov_model, output_path / "openvino_model.xml")
        
        config = {
            "model_type": "AutoencoderKL",
            "in_channels": vae.config.in_channels,
            "out_channels": vae.config.out_channels,
            "latent_channels": vae.config.latent_channels,
            "scaling_factor": vae.config.scaling_factor,
        }
        
        with open(output_path / "config.json", 'w') as f:
            json.dump(config, f, indent=2)
    
    print(f"✓ VAE Encoder saved")

def export_vae_decoder(vae, output_path):
    """Export VAE decoder"""
    print("Converting VAE Decoder...")
    
    vae.decoder.eval()
    vae.decoder = vae.decoder.float()
    vae.post_quant_conv = vae.post_quant_conv.float()
    
    dummy_input = torch.randn(1, 4, 64, 64, dtype=torch.float32)
    
    class VAEDecoderWrapper(torch.nn.Module):
        def __init__(self, decoder, post_quant_conv):
            super().__init__()
            self.decoder = decoder
            self.post_quant_conv = post_quant_conv
            
        def forward(self, latent):
            latent = self.post_quant_conv(latent)
            return self.decoder(latent)
    
    wrapper = VAEDecoderWrapper(vae.decoder, vae.post_quant_conv)
    
    with tempfile.NamedTemporaryFile(suffix=".onnx", delete=False) as tmp_file:
        torch.onnx.export(
            wrapper,
            dummy_input,
            tmp_file.name,
            input_names=["latent"],
            output_names=["sample"],
            dynamic_axes={
                "latent": {0: "batch_size", 2: "height", 3: "width"},
                "sample": {0: "batch_size", 2: "height", 3: "width"}
            },
            opset_version=14,
            do_constant_folding=True
        )
        
        ov_model = ov.convert_model(tmp_file.name)
        output_path.mkdir(parents=True, exist_ok=True)
        ov.save_model(ov_model, output_path / "openvino_model.xml")
        
        config = {
            "model_type": "AutoencoderKL",
            "in_channels": vae.config.in_channels,
            "out_channels": vae.config.out_channels,
            "latent_channels": vae.config.latent_channels,
            "scaling_factor": vae.config.scaling_factor,
        }
        
        with open(output_path / "config.json", 'w') as f:
            json.dump(config, f, indent=2)
    
    print(f"✓ VAE Decoder saved")

## 🚀 Step 4: Run Conversion

In [ ]:
print("Loading SD 1.5...")
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    variant="fp16",
    safety_checker=None
)
pipe = pipe.to("cuda")

print("Loading LoRA...")
lora_path = Path("lora_vietnamese/pytorch_lora_weights.safetensors")
pipe.load_lora_weights(str(lora_path.parent), weight_name=lora_path.name)

print("Fusing LoRA...")
pipe.fuse_lora(lora_scale=1.0)

print("Moving to CPU for export...")
pipe = pipe.to("cpu")

output_path = Path("vietnamese_ov")
output_path.mkdir(exist_ok=True)

print("Converting Text Encoder...")
export_text_encoder(pipe.text_encoder, pipe.tokenizer, output_path / "text_encoder")

print("Converting UNet...")
export_unet(pipe.unet, output_path / "unet")

print("Converting VAE Encoder...")
export_vae_encoder(pipe.vae, output_path / "vae_encoder")

print("Converting VAE Decoder...")
export_vae_decoder(pipe.vae, output_path / "vae_decoder")

print("Saving additional files...")
with tempfile.TemporaryDirectory() as tmp_dir:
    pipe.save_pretrained(tmp_dir)
    files_to_copy = [
        "tokenizer/merges.txt",
        "tokenizer/vocab.json",
        "tokenizer/special_tokens_map.json",
        "tokenizer/tokenizer_config.json",
        "scheduler/scheduler_config.json",
    ]
    for file_path in files_to_copy:
        src = Path(tmp_dir) / file_path
        dst = output_path / file_path
        if src.exists():
            dst.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(src, dst)

model_index = {
    "text_encoder": ["transformers", "CLIPTextModel"],
    "tokenizer": ["transformers", "CLIPTokenizer"],
    "unet": ["diffusers", "UNet2DConditionModel"],
    "vae": ["diffusers", "AutoencoderKL"],
    "scheduler": ["diffusers", "PNDMScheduler"],
    "safety_checker": None,
    "feature_extractor": None,
    "_class_name": "StableDiffusionPipeline",
    "_diffusers_version": "0.24.0"
}
with open(output_path / "model_index.json", 'w') as f:
    json.dump(model_index, f, indent=2)

print("DONE")

## 📦 Step 5: Zip và Download

In [ ]:
import shutil
from google.colab import files

print("Creating zip file...")
shutil.make_archive('vietnamese_ov', 'zip', 'vietnamese_ov')
print("✓ Zip created: vietnamese_ov.zip")

print("\nDownloading...")
files.download('vietnamese_ov.zip')
print("✓ Download started!")

print("\n" + "="*60)
print("DONE!")
print("="*60)
print("\nNext steps:")
print("1. Extract vietnamese_ov.zip")
print("2. Copy folder 'vietnamese_ov' to D:/NCKH_OpenVINO/models/")
print("3. Run: python app_enhanced.py")
print("4. Select 'Vietnamese LoRA' in dropdown")